**Decription:**
Script to solve a pipe-rotating puzzle for the Watch Cats problem in the CODE BLUE CTF 2018 Quals.

Basically, theres a water source in pipe segment 0, various joints of different shapes which connect certain segments and which can be rotated 90 degress at a time, and a set of goal segments. You must determine orientations for the different joints which allow the water be able to flow to all the goal segments.

Puzzles are in the following (dictionary) format:
- attempts: the max number of rotations allowed
- num_cables: the number of segments of pipe
- num_joints: the number of rotate-able pipe joints
- connections: list containing lists of the segments each joint connects to in order N,E,S,W (-1 if there is no segment in a direction)
- joint_types: encodes the style and initial rotation of each joint ( e.g. 0 -> |_ )
- goals: a list of segments that must be joined to the 0th segment to win

The required output is:
- The number of times each pipe joint should be rotated 90 degrees counter-clockwise

**Initial Plan:**
The intial plan was to implement some kind of dynamic programming approach to the problem, noting that:
- If S[i,j] is the set of rotation states of the joints which join pipe segments i and j
- then S[i,j] = Union{ S[i,k] n S[k,j] for k != i,j }

and then trying to find suitable constraints on k to avoid cyclic subproblem dependence. However I wasn't able to find such a constraint which made the subproblems non-cyclic and which would give a correct solution.


**Actual solution:**
Instead, we calculate the set of rotation states that connect a given path from the 0th segment to a goal segment, storing the set of states connecting all subpaths from 0 to avoid recalculation:
- S[*path*] = set of rotation states of the joints which make *path* connected
- where *path* starts at 0

For each goal, we find the union over all simple paths from 0 to the goal of the states which connect the path. Then take the intersection of these over all goals:
- States[*goal*] = Union{ S[*path*] for each *path* a simple path from 0 to *goal* }
- FinalStates = Intersection{ States[*goal*] for each *goal* in goals }

An additional filtering step gives us those rotation states which can be achieved in less than or exactly *attempts* rotations.

**Representations:**
- Storing the set of rotation states explicitly takes up way too much memory, so instead they are stored as a union of hyperplanes in the *num_joints*-dimensional space, where hyperplanes are represented as a pair of lists: one giving the *free* dimensions (i.e. those joints which can be in any rotation state they like), and one giving the rotations of those joints that are *fixed* (and the rotations for those that are free just being given as 0)

**Additional notes:**
- The given Stage 5 is actually unsolvable: joint 32 redirects the flow from 0 into at most one of two distinct connected components, each of which contains goals.
- The expectation of the CTF challenge was to actually break the verification program to get it to think that the stage had been solved. To this end I have also included an additional *debug* joint type which looks like a +.
- My hope was to find a solution to the stage if joint 32 were replaced by a +, then overflow the integer storing the direction that joint 32 is facing to make it negative. The C++ modulus operator preserves the sign of the first argument, so then instead of checking the configuration for joint 32, the verifier program would potentially check the config of the joint before it.

Read in and parse the stages

In [1]:
lines = []
with open('stage-descs') as f:
    lines = f.readlines()

In [2]:
lines

['STAGE 1:\n',
 '============================\n',
 "{'attempts': 1, 'goals': [1], 'num_goals': 1, 'joint_types': [5], 'num_joints': 1, 'connections': [[1, -1, 0, -1]], 'num_cables': 2}\n",
 '\n',
 'STAGE 2:\n',
 '============================\n',
 "{'attempts': 9, 'goals': [8], 'num_goals': 1, 'joint_types': [6, 3, 3, 2, 2, 3], 'num_joints': 6, 'connections': [[3, -1, 0, 1], [2, 1, -1, -1], [6, -1, 3, 4], [5, 4, 2, -1], [-1, -1, 6, 7], [8, 7, 5, -1]], 'num_cables': 9}\n",
 '\n',
 'STAGE 3:\n',
 '============================\n',
 "{'attempts': 8, 'goals': [12, 9, 13, 8, 14, 7], 'num_goals': 6, 'joint_types': [0, 5, 1, 5, 0, 9, 8, 4, 2, 6], 'num_joints': 10, 'connections': [[-1, 3, 0, 14], [-1, 0, 2, 1], [7, 1, 10, 12], [6, 11, 7, 8], [1, 4, 9, -1], [4, 5, -1, 13], [-1, 5, -1, 0], [10, -1, 6, 3], [-1, 12, 1, -1], [11, 12, -1, -1]], 'num_cables': 15}\n",
 '\n',
 'STAGE 4:\n',
 '============================\n',
 "{'attempts': 8, 'goals': [14, 4, 10, 1, 12, 19], 'num_goals': 6, 'joint_types'

In [3]:
import ast

stages = {}
i = 0
while i < len(lines):
    if (lines[i].startswith('STAGE')):
        name = lines[i][:-2]
        dictIn = ast.literal_eval(lines[i+2][:-1])
        stages[name] = dictIn
        i += 3
    else:
        i += 1

In [4]:
inputDict = stages['STAGE 3']

In [ ]:
# Stage 5 but with joint 32 replaced by a + shape
inputDict = {'attempts': 114514, 'goals': [1, 10, 23, 20, 24, 19, 25, 18, 40, 30, 36, 27, 38, 45], 'num_goals': 14, 'joint_types': [5, 6, 3, 3, 2, 2, 3, 0, 5, 1, 5, 0, 9, 8, 4, 2, 6, 7, 5, 8, 9, 8, 5, 3, 2, 7, 1, 8, 1, 8, 5, 3, 10, 4], 'num_joints': 34, 'connections': [[1, -1, 0, -1], [5, -1, 2, 3], [4, 3, -1, -1], [8, -1, 5, 6], [7, 6, 4, -1], [-1, -1, 8, 9], [10, 9, 7, -1], [-1, 14, 11, 25], [-1, 11, 13, 12], [18, 12, 21, 23], [17, 22, 18, 19], [12, 15, 20, -1], [15, 16, -1, 24], [-1, 16, -1, 11], [21, -1, 17, 14], [-1, 23, 12, -1], [22, 23, -1, -1], [37, -1, 45, 26], [45, 36, 47, 39], [47, 43, -1, 29], [30, -1, 40, 37], [38, 36, -1, 34], [29, 35, -1, 28], [30, 27, 44, 46], [-1, 41, 43, 33], [42, -1, 39, 32], [31, 44, -1, -1], [42, 39, 28, 31], [-1, -1, -1, 42], [31, 28, 37, 30], [27, -1, 26, 35], [42, 37, 39, -1], [48, 2, 11, 26], [0, -1, 48, -1]], 'num_cables': 49}

Use a NetworkX graph to be able to find all potential simple paths from 0 to a goal

In [5]:
import networkx as nx

G = nx.Graph()

for connection in inputDict['connections']:
    for i in range(1, 4):
        for j in range(0, i):
            if (connection[i] != -1 and connection[j] != -1):
                G.add_edge(connection[i],connection[j])

Given a joint of a given joint type, and two directions, we want to find the set of rotations which will make the joint join those two directions

In [6]:
# directions are 0:up, 1:right, 2:down, 3:left
# rotations are CCW
def rotationsSolving(jointType, dir1, dir2):
    rotations = {
        # up-right (and CW rotations)
        (0,0,1) : [0], (0,0,2) : [], (0,0,3) : [1], (0,1,2) : [3], (0,1,3) : [], (0,2,3) : [2],
        
        (1,0,1) : [1], (1,0,2) : [], (1,0,3) : [2], (1,1,2) : [0], (1,1,3) : [], (1,2,3) : [3],
        
        (2,0,1) : [2], (2,0,2) : [], (2,0,3) : [3], (2,1,2) : [1], (2,1,3) : [], (2,2,3) : [0],
        
        (3,0,1) : [3], (3,0,2) : [], (3,0,3) : [0], (3,1,2) : [2], (3,1,3) : [], (3,2,3) : [1],
        
        # up-down and rotation
        (4,0,1) : [], (4,0,2) : [0], (4,0,3) : [], (4,1,2) : [], (4,1,3) : [1], (4,2,3) : [],
        
        (5,0,1) : [], (5,0,2) : [1], (5,0,3) : [], (5,1,2) : [], (5,1,3) : [0], (5,2,3) : [],
        
        # up-right-down and rotations
        (6,0,1) : [0, 1], (6,0,2) : [0, 2], (6,0,3) : [1, 2], (6,1,2) : [0, 3], (6,1,3) : [1, 3], (6,2,3) : [2, 3],
        
        (7,0,1) : [1, 2], (7,0,2) : [1, 3], (7,0,3) : [2, 3], (7,1,2) : [1, 0], (7,1,3) : [0, 2], (7,2,3) : [3, 0],
        
        (8,0,1) : [2, 3], (8,0,2) : [2, 0], (8,0,3) : [3, 0], (8,1,2) : [2, 1], (8,1,3) : [3, 1], (8,2,3) : [0, 1],
        
        (9,0,1) : [3, 0], (9,0,2) : [3, 1], (9,0,3) : [0, 1], (9,1,2) : [3, 2], (9,1,3) : [0, 2], (9,2,3) : [1, 2],
        
        # debug type : up-right-down-left
        (10,0,1) : [0,1,2,3], (10,0,2) : [0,1,2,3], (10,0,3) : [0,1,2,3],
        (10,1,2) : [0,1,2,3], (10,1,3) : [0,1,2,3], (10,2,3) : [0,1,2,3]
    }
    return set(rotations[(jointType, min(dir1,dir2), max(dir1,dir2))])
    

We want to be able to find the intersection of two unions of hyperplanes

In [7]:
def intersection(s1, s2):
    ret = set()
    # For each hyperplane in the first set
    for (fr1,fx1) in s1:
        # And each hyperplane in the second set
        for (fr2,fx2) in s2:
            # Construct a potential intersection hyperplane
            fr = [i for i in range(len(fx1))]
            fx = [0 for i in range(len(fx1))]
            empty = False
            # Fix the value of the non-free directions
            for i in range(len(fx1)):  
                if ((not i in fr1) and (not i in fr2)):
                    # If the hyperplanes are both fixed in one direction,
                    #  but not at the same value, then they don't intersect
                    if (fx1[i] != fx2[i]):
                        empty = True
                        break
                    else:
                        fr.remove(i)
                        fx[i] = fx1[i]
                elif (not i in fr1):
                    fr.remove(i)
                    fx[i] = fx1[i]
                elif (not i in fr2):
                    fr.remove(i)
                    fx[i] = fx2[i]
            if (not empty):
                ret.add((tuple(fr),tuple(fx)))
    return ret

The main algorithm calculates the set of rotation states connecting a given path which starts at 0

In [8]:
# All states connect the path (0)
pathSolutions = { (0,) : set([(tuple([i for i in range(inputDict['num_joints'])]),tuple([0 for i in range(inputDict['num_joints'])]))]) }
# Keep ahold of the states which connect two adjacent segments
pairSolutions = {}
def solutionStates(path):
    states = set()
    pathStart = 0
    
    # First check for the longest subpath which has already been calculated
    for i in range(len(path)):
        if (path[:-i] in pathSolutions.keys()):
            states = pathSolutions[path[:-i]]
            pathStart = len(path) - 1 - i
            break
            
    # Now for each step of this path that hasn't already been calculated
    for i in range(pathStart+1, len(path)):
        
        # Give up if this path is impossible to join up
        if (states == set()):
            break
            
        stepStates = set()
        
        # If the set of states that join two adjacent segments is known, use it
        if ((min(path[i],path[i-1]),max(path[i],path[i-1])) in pairSolutions.keys()):
            stepStates = pairSolutions[(min(path[i],path[i-1]),max(path[i],path[i-1]))]
            
        # Otherwise, calculate it, store it, and use it
        else:
            for conInd in range(len(inputDict['connections'])):
                if (path[i-1] in inputDict['connections'][conInd] and path[i] in inputDict['connections'][conInd]):
                    solvingRotations = rotationsSolving(inputDict['joint_types'][conInd],
                                                        inputDict['connections'][conInd].index(path[i-1]),
                                                        inputDict['connections'][conInd].index(path[i]))
                    stepStates = stepStates.union(
                        set([(
                                tuple([i for i in range(inputDict['num_joints']) if i != conInd]),
                                tuple([(sol if (i == conInd) else 0) for i in range(inputDict['num_joints'])])
                            ) for sol in solvingRotations])
                    )
            pairSolutions[(min(path[i],path[i-1]),max(path[i],path[i-1]))] = stepStates
            
        # Cut down to states that connect both the path so far as well as this next step
        states = intersection(states, stepStates)
        
        # Store the solution for this subpath with the extra step
        pathSolutions[path[:i+1]] = states
        
    # Return the set of solution states for the whole path
    return states

Finally, we calculate the set of all rotation states that, for each of the goals, connects at least one path from 0 to that goal

In [9]:
# Initially we suppose that all states work
final_states = set([(tuple([i for i in range(inputDict['num_joints'])]),tuple([0 for i in range(inputDict['num_joints'])]))])
for goal in inputDict['goals']:
    # Find the set of states that allow the water to flow from 0 to the goal along at least one path
    states = set()
    for path in nx.all_simple_paths(G, 0, goal):
        states = states.union(solutionStates(tuple(path)))
    # Find the set of states that allow the water to flow from 0 to every goal considered so far
    final_states = intersection(final_states, states)

In [10]:
final_states

{((7,), (2, 0, 1, 0, 3, 0, 1, 0, 1, 0)),
 ((7,), (2, 0, 1, 0, 3, 0, 1, 0, 1, 1)),
 ((7,), (2, 0, 1, 0, 3, 0, 3, 0, 1, 0)),
 ((7,), (2, 0, 1, 0, 3, 0, 3, 0, 1, 1)),
 ((7,), (2, 0, 2, 0, 3, 0, 1, 0, 1, 0)),
 ((7,), (2, 0, 2, 0, 3, 0, 1, 0, 1, 1)),
 ((7,), (2, 0, 2, 0, 3, 0, 3, 0, 1, 0)),
 ((7,), (2, 0, 2, 0, 3, 0, 3, 0, 1, 1))}

Now just give the ones which require less than or exactly *attempts* inputs to the program

In [11]:
def filterForAttempts(states):
    return set(filter(lambda x: sum(x[1]) <= inputDict['attempts'], states))

In [12]:
filterForAttempts(final_states)

{((7,), (2, 0, 1, 0, 3, 0, 1, 0, 1, 0))}

Generate a command list to feed to the program

In [ ]:
sols = [[0],[0,0,1,1,1,3,5,5,5],[0,0,2,4,4,4,6,8],[3,4,4,6,6,6,10,13]]
with open('commands.txt','w') as f:
    for sol in sols:
        for num in sol:
            f.write(str(num)+'\n')
    for i in range(32767+2):
        f.write(str(32)+'\n')